In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Overview

This is a generalized notebook for computing grade statistics from the Ted Grade Center.

In [1]:
#The usual imports
import math
import glob
import os
from collections import OrderedDict

from pandas import read_csv
import numpy as np

from pymatgen.util.plotting_utils import get_publication_quality_plot
from monty.string import remove_non_ascii

import prettyplotlib as ppl
from prettyplotlib import brewer2mpl
import matplotlib.pyplot as plt

colors = brewer2mpl.get_map('Set1', 'qualitative', 8).mpl_colors

import datetime

%matplotlib inline

print("Last updated on %s" % datetime.datetime.now())

Last updated on 2016-03-21 17:25:16.683147


Load data from exported CSV from Ted Full Grade Center. Some sanitization is performed to remove non-ascii characters and cruft.

In [5]:
files = glob.glob(os.environ["NANO106GC"])
latest = sorted(files)[-1]
d = read_csv(latest)
d.columns = [remove_non_ascii(c) for c in d.columns]
d.columns = [c.split("[")[0].strip().strip("\"") for c in d.columns]
d["Weighted Total"] = [float(s.strip("%")) for s in d["Weighted Total"]]

Define lower grade cutoffs in terms of number of standard deviations from mean.

In [7]:
grade_cutoffs = OrderedDict()
grade_cutoffs["A+"] = 1.5
grade_cutoffs["A"] = 1.0
grade_cutoffs["A-"] = 0.75
grade_cutoffs["B+"] = 0.5
grade_cutoffs["B"] = 0
grade_cutoffs["B-"] = -0.5
grade_cutoffs["C+"] = -0.75
grade_cutoffs["C"] = -1
grade_cutoffs["C-"] = -1.5
grade_cutoffs["F"] = float("-inf")

print("The cutoffs are:")
for k, v in grade_cutoffs.items():
    print(u"%s: > μ + %.2f σ" % (k, v))

The cutoffs are:
A+: > μ + 1.50 σ
A: > μ + 1.00 σ
A-: > μ + 0.75 σ
B+: > μ + 0.50 σ
B: > μ + 0.00 σ
B-: > μ + -0.50 σ
C+: > μ + -0.75 σ
C: > μ + -1.00 σ
C-: > μ + -1.50 σ
F: > μ + -inf σ


Define some general functions for computing grade statistics.

In [8]:
def bar_plot(dframe, data_key, offset=0, annotate=True):
    """
    Creates a historgram of the results.
    
    Args:
        dframe: DataFrame which is imported from CSV.
        data_key: Specific column to plot
        offset: Allows an offset for each grade. Defaults to 0.
        
    Returns:
        dict of cutoffs, {grade: (lower, upper)}
    """
    data = dframe[data_key]
    d = filter(lambda x: (not np.isnan(x)) and x != 0, list(data))
    heights, bins = np.histogram(d, bins=20, range=(0, 100))
    bins = list(bins)
    bins.pop(-1)
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(1)
    ppl.bar(ax, bins, heights, width=5, color=colors[0], grid='y')
    plt = get_publication_quality_plot(12, 8, plt)
    plt.xlabel("Score")
    plt.ylabel("Number of students")
    #print len([d for d in data if d > 90])
    mean = np.mean(d)
    sigma = np.std(d)
    maxy = np.max(heights)
    prev_cutoff = 100
    cutoffs = {}
    grade = ["A", "B+", "B", "B-", "C+", "C", "C-", "F"]
    for grade, cutoff in grade_cutoffs.items():
        if cutoff == float("-inf"):
            cutoff = 0
        else:
            cutoff = max(0, mean + cutoff * sigma) + offset
            if annotate:
                plt.plot([cutoff] * 2, [0, maxy], 'k--')
                plt.annotate("%.1f" % cutoff, [cutoff, maxy - 1], fontsize=18, horizontalalignment='left', rotation=45)
        n = len([d for d in data if cutoff <= d < prev_cutoff])
        #print "Grade %s (%.1f-%.1f): %d" % (grade, cutoff, prev_cutoff, n)
        if annotate:
            plt.annotate(grade, [(cutoff + prev_cutoff) / 2, maxy], fontsize=18, horizontalalignment='center')
        cutoffs[grade] = (cutoff, prev_cutoff)
        prev_cutoff = cutoff
    
    plt.ylim([0, maxy * 1.1])
    plt.annotate("$\mu = %.1f$\n$\sigma = %.1f$\n$max=%.1f$" % (mean, sigma, data.max()), xy=(10, 7), fontsize=30)
    title = data_key.split("[")[0].strip()
    plt.title(title, fontsize=30)
    plt.tight_layout()
    plt.savefig("%s.png" % title)
    return cutoffs


def assign_grades(d, column_name, cutoffs, offset):
    def compute_grade(pts):
        for g, c in cutoffs.items():
            if c[0] < pts + offset <= c[1]:
                return g

    d["Final_Assigned_Egrade"] = map(compute_grade, d[column_name])
    d.to_csv("Overall grades.csv")

# Problem Sets

In [18]:
#cutoffs = bar_plot(d, "PS1", annotate=True)

In [19]:
#cutoffs = bar_plot(d, "PS2", annotate=True)

In [20]:
#cutoffs = bar_plot(d, "PS3", annotate=True)

In [21]:
#cutoffs = bar_plot(d, "PS4", annotate=True)

In [22]:
#cutoffs = bar_plot(d, "PS5", annotate=True)

# Exams

In [23]:
#cutoffs = bar_plot(d, "Mid-term 1", annotate=True)

In [24]:
#cutoffs = bar_plot(d, "Mid-term 2", annotate=True)

In [25]:
#cutoffs = bar_plot(d, "Final", annotate=True)

# Overall grade

In [26]:
#cutoffs = bar_plot(d, "Weighted Total", annotate=True)

In [15]:
# The command below is used to generate the overall grade assignments for all students and dump it into a CSV file.
# assign_grades(d, "Weighted Total", cutoffs)